# Interacting with LLMs from OpenAI through its API

### Import initialization  

In [ ]:
import openai
import os
from dotenv import load_dotenv


Load environment variables from .env file if it exists

In [ ]:
load_dotenv("./../../.env")

# Open AI API Key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
completion = openai.ChatCompletion.create( 
  model="gpt-3.5-turbo", 
  messages=[ 
    {"role": "system", "content": "You are a helpful assistant."}, 
    {"role": "user", "content": "Hello!"} 
  ] 
) 

print(completion.choices[0].message) 

# Interacting with Transformers through HuggingFace's API

**Important** It is recommended that you execute this notebook on a GPU-enabled machine. You can do this from Google Colab as follows:

1. Visit https://colab.research.google.com/
2. Click on the GitHub tab
3. Paste the URL to the repository or notebook. You will need to select A100 GPU runtime. 

https://github.com/PacktPublishing/Building-Natural-Language-Pipelines


![](./images/colab.png)

![](./images/runtime.png)

Initializing model access and tokenizer.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigscience/mt0-small"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

Tokenization: Before feeding text to a model, you need to tokenize it. Using the above tokenizer, you can tokenize a sentence like this:

In [ ]:
inputs = tokenizer.encode("Translate to English: Je t’aime.", return_tensors="pt")

Model Inference: Once tokenized, you can feed the input tensors to the model to get predictions:

In [ ]:
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

# Interacting with LLMs from various providers using Haystack 

### Prompt node initialization - no base instructions

In [ ]:
from haystack.nodes import PromptTemplate, AnswerParser, PromptNode

In [ ]:
# Create a prompt node
openai_api_key = os.getenv("OPENAI_API_KEY")

pn = PromptNode("gpt-3.5-turbo", 
                api_key=openai_api_key, 
                model_kwargs={"stream":False}, 
                max_length=1000)

# Create a prompt
prompt = "What are the three most interesting things about Berlin? Be brief in your answer."


In [ ]:
# using a prompt template
tourist_recommendation_prompt = PromptTemplate(
    prompt="""You are a helpful assistant to a tourists looking find places to\
            visit. You provide information on wether, tourist attractions\
            The tourist may or may not know what city they are interested in\
            so you recommend based on their request.\
            If the request is vague, ask the tourist to provide more information\
            such as the type of wether they want, what kind of experience they want\
            weather it is family friendly or not. If the tourist is interested in\
            a specific city, ask them to specify the city.\
            and cost of visiting" You answer:""",
    output_parser=AnswerParser(),
)
pn = PromptNode("gpt-3.5-turbo", 
                api_key=openai_api_key, 
                model_kwargs={"stream":False},
                default_prompt_template=tourist_recommendation_prompt)

In [ ]:
pn.run("I am looking for a place with tropical wether in America")

Prompting LLMs from HF

In [ ]:
from haystack.nodes import PromptNode

# Initalize the node passing the model:
prompt_node = PromptNode(model_name_or_path="google/flan-t5-xl")

# Go ahead and ask a question:
prompt_node("What is the best city in Europe to live in?")
